In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_images_x = pd.read_pickle('/kaggle/input/modified-mnist/train_max_x')
train_images_y = pd.read_csv('/kaggle/input/modified-mnist/train_max_y.csv')
test_images = pd.read_pickle('/kaggle/input/modified-mnist/test_max_x')



In [ ]:
train_images_x.shape


In [ ]:
import cv2
import matplotlib.pyplot as plt


# Gaussian Filter

In [ ]:
def filter_image(img):
    image = np.array(img, dtype=np.uint8)
    image = (255-image)
    kernel = np.ones((1,1),np.uint8)

#     plt.imshow(image, 'gray', vmin=0, vmax=255)
#     plt.show()
    
#     gaus = cv2.GaussianBlur(image, (1,1), 0)
#     plt.imshow(gaus, 'gray', vmin=0, vmax=255)
#     plt.show()
    
    median = cv2.medianBlur(image, 1)
#     plt.imshow(median, 'gray', vmin=0, vmax=255)
#     plt.show()
    
    thresh = cv2.threshold(median.copy(), 60, 255, cv2.THRESH_BINARY)[1]
    
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

#     plt.imshow(thresh, 'gray', vmin=0, vmax=255)
#     plt.show()
    return opening

## Pretty Print images for Report

In [ ]:
f, ax = plt.subplots(2, 2)
ax[0,0].imshow(train_images_x[25], cmap='gray')
ax[0,0].axis('off')
ax[0,1].imshow(train_images_x[55], cmap='gray')
ax[0,1].axis('off')
ax[1,0].imshow(train_images_x[75], cmap='gray')
ax[1,0].axis('off')
ax[1,1].imshow(train_images_x[85], cmap='gray')
ax[1,1].axis('off')


## Filtering all images

In [ ]:
x_train_imgs = np.full(train_images_x.shape, 0)
x_test_imgs = np.full(test_images.shape, 0)

In [ ]:


for i in range (0, len(train_images_x)):
    x_train_imgs[i] = filter_image(train_images_x[i])
    
for i in range (0, len(test_images)):
    x_test_imgs[i] = filter_image(test_images[i])



In [ ]:
import torch
from torchvision.utils import make_grid

random_sel = np.random.randint(len(train_images_x), size=8)

grid = make_grid(torch.Tensor((train_images_x[random_sel]/255.).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')
print(*list(train_images_y.iloc[random_sel, 0].values), sep=', ')

plt.savefig('images.png')

### invert the colors


In [ ]:
grid = make_grid(torch.Tensor(((255-train_images_x[random_sel])/255.).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')


In [ ]:
inv_images = 255-train_images_x[random_sel]

### apply a median filter

In [ ]:
for i in range (0, len(inv_images)):
    med_images[i] = cv2.medianBlur(inv_images[i].copy(), 1)


grid = make_grid(torch.Tensor((med_images/255.).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')
plt.savefig('median_blur.png')

In [ ]:
thresh_images = med_images
for i in range (0, len(thresh_images)):
    thresh_images[i] = cv2.threshold(med_images[i].copy(), 30, 255, cv2.THRESH_BINARY)[1]



grid = make_grid(torch.Tensor((thresh_images/255.).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')
plt.savefig('thresh_imgs.png')

In [ ]:
morph_images = thresh_images
kernel = np.ones((3,3),np.uint8)

for i in range (0, len(thresh_images)):
    morph_images[i] = cv2.dilate(thresh_images[i].copy(), kernel)



grid = make_grid(torch.Tensor((morph_images/255.).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')
plt.savefig('morph_imgs.png')

In [ ]:
grid = make_grid(torch.Tensor((x_train_imgs[random_sel]/255.).reshape((-1,128,128))).unsqueeze(1), nrow=8)
plt.rcParams['figure.figsize'] = (16,2)
plt.imshow(grid.numpy().transpose((1,2,0)))
plt.axis('off')

In [ ]:
for i in range (0, 20):
    plt.imshow(x_train_imgs[i], 'gray', vmin=0, vmax=255)
    plt.show()